In [1]:
import json
import numpy as np
import pandas as pd
import re

In [18]:
def get_list_of_dicts(fname): return [json.loads(i) for i in open(fname, "rt")]

In [19]:
raw_data = get_list_of_dicts("../data/reviews_Amazon_Instant_Video_5.json")
raw_data[0]

{'asin': 'B000H00VBQ',
 'helpful': [0, 0],
 'overall': 2.0,
 'reviewText': "I had big expectations because I love English TV, in particular Investigative and detective stuff but this guy is really boring. It didn't appeal to me at all.",
 'reviewTime': '05 3, 2014',
 'reviewerID': 'A11N155CW1UV02',
 'reviewerName': 'AdrianaM',
 'summary': 'A little bit boring for me',
 'unixReviewTime': 1399075200}

In [22]:
from IPython.display import display
data = pd.DataFrame(raw_data).loc[:, ["reviewerID", "reviewText", "asin", "overall"]]
user_item_revew = data.drop("reviewText", axis=1)
user_reviews = pd.pivot_table(data, index=["reviewerID", "asin"], aggfunc=lambda x: x)
user_scores = pd.pivot_table(data, index=["reviewerID", "asin"])
movie_reviews = pd.pivot_table(data, index=["asin", "reviewerID"], aggfunc=lambda x: x)
movie_scores = pd.pivot_table(data, index=["asin", "reviewerID"])
users = pd.concat([user_reviews, user_scores], axis=1)
movies = pd.concat([movie_reviews, movie_scores], axis=1)
users[:2]
print(users.loc["A11N155CW1UV02", "B008FWHGYS"]["reviewText"])

Not much at least, but I didn't get interested in finish it so I guess that says a lot about the movie. The characters didn't make it for me, kind of boring.


In [ ]:
def add_user_reviews(x):
    x["userReviews"] = user_reviews.loc[x["reviewerID"]].drop(x["asin"])
    x["movieReviews"] = movie_reviews.loc[x["asin"]].drop(x["reviewerID"])
    return x
dat = user_item_revew.apply(add_user_reviews, axis=1)

In [ ]:
with open("../data/unembedded_grouped_reviews.txt", "w") as f:
    f.write(dat.to_json())